In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from activ import load_data
from PyUoI.UoINMF import UoINMF
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from activ.data_normalization import data_normalization
from scipy.spatial.distance import pdist
from sklearn.model_selection import KFold
import h5py

In [13]:
data = load_data()
data_bm = data_normalization(data.biomarkers, 'positive')
knum = 2
kf = KFold(n_splits=knum)
# eps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
eps = [0.2]
# minsamples = np.arange(20,80,5)
minsamples = np.arange(20,30,5)
error_mat = np.zeros((len(eps), len(minsamples), knum))
k_mat = np.zeros((len(eps), len(minsamples), knum))
W_mat = np.zeros((len(eps), len(minsamples), knum))

In [20]:
for cv, (train_index, test_index) in enumerate(kf.split(data_bm)):
    train, test = data_bm[train_index], data_bm[test_index]
    train_bm = train
    test_bm = test
    for ii,e in enumerate(eps):
        for jj,m in enumerate(minsamples):
            db = DBSCAN(eps=e, min_samples=m)
            uoinmf = UoINMF(ranks=list(range(2,20)),dbscan=db)
            decomp = uoinmf.fit(train_bm)
            weight = uoinmf.fit_transform(train_bm)
            print(weight.shape)
            H = decomp.components_
            k = H.shape[0]
            W = decomp.transform(test_bm, reconstruction_err=True)
            error = np.linalg.norm(test_bm-W@H)
            error_mat[ii,jj,cv] = error
            k_mat[ii,jj,cv] = k
#             plt.figure()
#             plt.matshow(weight, cmap='gray_r')


(123, 1)
(123, 1)
(124, 1)
(124, 1)


In [32]:
f = h5py.File('optimize_k_mat_bm_prac.h5', 'w')
f.create_dataset('error', data=error_mat)
f.create_dataset('k', data=k_mat)
f.create_dataset('eps', data=eps)
f.create_dataset('minsamples', data=minsamples)
f.create_dataset('W', data=W_mat)
f.close()